<a href="https://colab.research.google.com/github/IZ001/Stat-Machine-Learning/blob/main/Feature%20importance%20task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.linear_model as skl_lm
import sklearn.preprocessing as skl_pre
import sklearn.model_selection as skl_ms
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

from google.colab import files
uploaded = files.upload()

import io
data = pd.read_csv(io.BytesIO(uploaded['train.csv']), na_values = '?').dropna().reset_index(drop=True)
# Dataset is now stored in a Pandas Dataframe

Saving train.csv to train (1).csv


In [11]:
print(data['Lead'])

0       Female
1         Male
2         Male
3         Male
4         Male
         ...  
1034      Male
1035    Female
1036      Male
1037    Female
1038      Male
Name: Lead, Length: 1039, dtype: object


In [12]:
data['Lead'] = np.where(data['Lead'] == 'Female', 1, 0)


In [13]:
np.sum(data['Lead'])

254

In [52]:
# Evaluate performance of logistic regression model

#Input features
features = ['Difference in words lead and co-lead','Number words female', 'Number words male',  'Age Co-Lead', 'Number of female actors', 'Gross', 'Year', 'Total words', 'Number of words lead', 'Number of male actors',  'Mean Age Male', 'Mean Age Female','Age Lead']
#features = ['Gross']


#best subset of features = ['Total words', 'Age Co-Lead', 'Number of female actors', 'Difference in words lead and co-lead', 'Number of words lead', 'Age Lead', 'Number words female', 'Number of male actors']
X = data[features]
y = data['Lead']

n_fold = 10
pc_auc_n2 =np.zeros((n_fold))
roc_auc_n2 = np.zeros((n_fold))
misclassification_n2 = np.zeros((n_fold))
true_positive_rate = []
false_positive_rate = []
f1_rate = []
tr_pos = 0
fal_pos = 0

positive_class = 1
negative_class = 0

cv = skl_ms.KFold(n_splits = n_fold, random_state = 1, shuffle = True)

for i, (train_index, val_index) in enumerate(cv.split(X)):
  X_train, X_val = X.iloc[train_index], X.iloc[val_index]
  y_train, y_val = y.iloc[train_index], y.iloc[val_index]

  P = np.sum(y_val == positive_class)
  N = np.sum(y_val == negative_class)
  
  model = skl_lm.LogisticRegression(solver ='liblinear')
  #, C = 18.31792545756584, penalty ='l1')

  scaler = skl_pre.StandardScaler().fit(X_train)
  model.fit(scaler.transform(X_train[features]), y_train)
  predict_prob = model.predict_proba(scaler.transform(X_val))

  # calculate the metrics
  precision, recall, _ = precision_recall_curve(y_val, predict_prob[:,1])

  pc_auc_n2[i] = auc(recall, precision)
  roc_auc_n2[i] = roc_auc_score(y_val, predict_prob[:,1])  

  tr_rate = 0.5
  prediction = np.where(predict_prob[:,1] > tr_rate, 1, 0)
  print(prediction)
  misclassification_n2[i] = (np.mean(prediction != y_val))
  tr_pos = np.sum((prediction == y_val)&(prediction == positive_class))
  fal_pos = np.sum((prediction != y_val)&(prediction == positive_class))
  #print(tr_pos)
  #print(fal_pos)
  true_positive_rate.append(tr_pos/P)
  print(true_positive_rate)
  false_positive_rate.append(fal_pos/N)
  f1 = f1_score(y_val, prediction, average='binary')
  f1_rate.append(f1) 

print('ROC AUC', np.average(roc_auc_n2))
print('PC AUC', np.average(pc_auc_n2))
print('Error', np.average(misclassification_n2))
print('TPR', np.average(true_positive_rate))
print('FPR', np.average(false_positive_rate))
print('f1 coef', np.average(f1_rate))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.0, 0.0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.0, 0.0, 0.0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.0, 0.0, 0.0, 0.0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 